In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn import *
from sklearn.metrics import *
from itertools import islice
from sklearn.preprocessing import normalize
from tensorflow.keras.models import load_model

from sklearn.model_selection import StratifiedKFold
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-27 16:44:51.634560: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 16:44:51.922575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 16:44:51.922608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 16:44:51.975615: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-27 16:44:52.077941: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15094084657765773426
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6731746705645700252
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-27 16:44:54.522684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:54.688199: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:54.688260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:55.106900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:55.106945: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

X_train = train_values[:,:-2]
X_test = test_values[:,:-2]

X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

y_train = train_values[:,-2]
y_test = test_values[:,-2]

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

(38751, 277)
(72794, 277)


In [4]:
cnnmodel = load_model('./models/cnnmodel_grad.h5')

2024-03-27 16:44:56.875024: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:56.875088: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:56.875104: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:56.875206: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-27 16:44:56.875222: I external/local_xla/xla/stream_executor

In [8]:
feature_importance_df = pd.DataFrame()

for class_num in range(1,7):

    
    N_test = test_values[test_values[:,-2]==class_num]

    if class_num == 1:
                X_test = N_test[:10000,:-2]
    else:
                X_test = N_test[:,:-2]
    y_test = N_test[:len(N_test),-2]

    X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
    y_test1 = to_categorical(y_test)
    cnnpredictions = cnnmodel.predict(X_test1, verbose=1)

    cnn_predict=np.argmax(cnnpredictions,axis=1)
    cnn_actual_value=np.argmax(y_test1,axis=1)


    cnnmodel.layers[-1].activation = None

    grad_model = tf.keras.models.Model([cnnmodel.inputs], [cnnmodel.get_layer('last_conv_layer').output, cnnmodel.output])

    with tf.GradientTape() as tape:
        last_conv_layer_features, y_predictions = grad_model(X_test1)
    grads = tape.gradient(y_predictions, last_conv_layer_features)

    alpha_values = tf.reduce_mean(grads, axis = 1)

    gradcam = tf.linalg.matmul(last_conv_layer_features, alpha_values[..., tf.newaxis])


    gradcam = gradcam.numpy()
    gradcam = np.abs(gradcam)
    gradcam_norm = (gradcam - np.min(gradcam, axis = 1, keepdims = True)) / (np.max(gradcam, axis = 1, keepdims = True) - 
                                                            np.min(gradcam, axis = 1, keepdims = True))
    y_predictions = tf.nn.softmax(y_predictions)
    y_predictions = y_predictions.numpy()


    def means_of_slices(iterable, slice_size):
        iterator = iter(iterable)
        while True:
            slice = list(islice(iterator, slice_size))
            if slice:
                yield np.sum(slice)/len(slice)
            else:
                return

    cam_per_segment = []
    for i in range(X_test1.shape[0]):
        means = list(means_of_slices(gradcam_norm[i], 12))
        cam_per_segment.append(means)
    cam_per_segment = np.array(cam_per_segment)
    print(cam_per_segment.shape)


    ecg_norm = (X_test1[20, :] - X_test1[20, :].min()) / (X_test1[20, :].max() - X_test1[20, :].min())

    mean_cam_norm = cam_per_segment.mean(axis=0)

    feature_importance_normalized = (mean_cam_norm - mean_cam_norm.min()) / (mean_cam_norm.max() - mean_cam_norm.min())

    indices_sort = np.argsort(-1 * mean_cam_norm)

    slices = np.arange(1, 24)

    fig, ax = plt.subplots(1, 2, figsize = (15, 4))
    sns.barplot(x=np.arange(23), y=feature_importance_normalized[indices_sort], ax=ax[0],palette="pastel")
    ax[0].set_title('Average Grad-CAM values for CNN model per segment over Class: ' + str(class_num))
    ax[0].set_xticks(np.arange(0,23))
    ax[0].set_xticklabels(slices[indices_sort])
    ax[0].set_xlabel('Slice')
    ax[0].set_ylabel('Average Grad-CAM values')

    ax[1].plot(np.arange(len(ecg_norm)), ecg_norm, label='ECG data')
    ax[1].plot(np.repeat(feature_importance_normalized,12), label = 'Average Grad-CAM values')
    ax[1].set_title('Average Grad-CAM values for CNN model per segment over Class: ' + str(class_num))
    ax[1].set_xlabel('Time')
    ax[1].set_ylabel('ECG signal / Average Grad-CAM values')
    ax[1].legend()
    image_name = 'output_pics/Average Grad-CAM values for CNN model per segment over Class: ' + str(class_num)+'.jpg'
    fig.savefig(image_name, dpi = 400)

    feature_importance_df['class_' + str(class_num)] = feature_importance_normalized
feature_importance_df.to_csv('./output_csv/GRADCAM_weights_CNN_model.csv', index=False)


2024-03-27 16:45:35.906155: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-27 16:45:36.023310: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-27 16:45:36.148578: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 [==============================] - 2s 3ms/step


2024-03-27 16:45:37.813340: W tensorflow/core/kernels/gpu_utils.cc:54] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2024-03-27 16:45:38.626420: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-27 16:45:38.679106: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-27 16:45:38.679160: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator

ResourceExhaustedError: Exception encountered when calling layer 'conv1d' (type Conv1D).

{{function_node __wrapped__BiasAdd_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[10000,275,396] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BiasAdd] name: 

Call arguments received by layer 'conv1d' (type Conv1D):
  • inputs=tf.Tensor(shape=(10000, 275, 1), dtype=float32)